# 🚀 KcELECTRA MTL (v3) — 추론 및 성능 검증

학습된 `best_mtl.pt` 모델을 로드하여 **테스트셋 성능을 검증**하고, **새로운 텍스트에 대한 추론**을 수행합니다.

**실행 방법:**
1.  **[필수]** **1번 셀**을 먼저 실행하여 드라이브를 마운트하고 학습 환경과 동일한 라이브러리를 설치합니다.
2.  1번 셀 실행 완료 시 **런타임이 자동으로 재시작**됩니다.
3.  런타임 재시작 후, **2번 셀부터** 마지막 셀까지 순서대로 실행합니다.

## 1. 드라이브 마운트 및 환경 구성

학습 스크립트(`kcelectra_mtl_a100_turbo_v3.py`)와 동일한 환경을 구성합니다.

In [ ]:
# ⬇️ Google Drive & deps
from google.colab import drive
drive.mount('/content/drive')

# === Clean & Pin: Colab 기본 스택과 호환되는 조합으로 맞춥니다. ===
import sys, subprocess, os

def pip(*args):
    print(">>", " ".join(args))
    subprocess.check_call([sys.executable, "-m", "pip", *args])

# 1) 충돌나는 핵심 패키지 제거
pip("uninstall", "-y", "pyarrow", "pandas", "numpy", "fastparquet", "scikit-learn")

# 2) 호환 조합으로 설치
pip("install", "-q", "numpy==2.0.2", "pandas==2.2.2", "pyarrow==16.1.0", "scikit-learn==1.5.2")

# 3) NLP 스택(Transformers/Accelerate)은 '의존성 자동 업그레이드'를 막습니다.
pip("install", "-q", "--no-deps", "transformers>=4.44.0", "accelerate>=0.33.0", "tqdm>=4.66.4", "pandas==2.2.2")

print("\n✅ Reinstall done. Now restarting kernel to finalize...")
os.kill(os.getpid(), 9)  # 🔁 런타임 강제 재시작

Mounted at /content/drive
>> uninstall -y pyarrow pandas numpy fastparquet scikit-learn
>> install -q numpy==2.0.2 pandas==2.2.2 pyarrow==16.1.0 scikit-learn==1.5.2
>> install -q --no-deps transformers>=4.44.0 accelerate>=0.33.0 tqdm>=4.66.4 pandas==2.2.2


--- (런타임 재시작 후 여기부터 실행) ---

## 2. 기본 설정 및 임포트

In [2]:
import torch, platform, warnings
import os, random, numpy as np, pandas as pd
from pathlib import Path
from tqdm.auto import tqdm
from torch.amp import autocast
import torch.nn as nn
from torch.utils.data import DataLoader
from sklearn.metrics import f1_score, classification_report, confusion_matrix
from transformers import AutoTokenizer, AutoModel

warnings.filterwarnings('ignore')
print("PyTorch:", torch.__version__, "| CUDA:", torch.version.cuda)
if torch.cuda.is_available():
    print("GPU:", torch.cuda.get_device_name(0))
print("Python:", platform.python_version())

# ⚙️ Config (학습 스크립트와 동일하게 설정)
SEED = 42
random.seed(SEED); np.random.seed(SEED); torch.manual_seed(SEED)
if torch.cuda.is_available():
    torch.cuda.manual_seed_all(SEED)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Device:", device)

# 경로 설정
DATA_DIR  = Path("/content/drive/MyDrive/project1/Processed_Data/MTL_ready_v3")  # ✅ 테스트 데이터 로드용
SAVE_DIR  = Path("/content/drive/MyDrive/project1/kcelectra_mtl_ckpt_v3") # ✅ 모델 로드용
CACHE_DIR = Path("/content/kcelectra_mtl_cache") # ✅ 테스트 청크 캐시용
CACHE_DIR.mkdir(parents=True, exist_ok=True)

# AMP 설정
bf16_ok = torch.cuda.is_available() and torch.cuda.is_bf16_supported()
amp_dtype = torch.bfloat16 if bf16_ok else torch.float16
if torch.cuda.is_available():
    torch.backends.cuda.matmul.allow_tf32 = True

# 하이퍼파라미터 (모델 구조/토크나이저 정의에 필요한 부분만)
BATCH_SIZE = 160 # 평가 시 배치 크기 (OOM 시 128, 96 등으로 감소)
NUM_WORKERS = 4
PREFETCH = 4
PIN_MEMORY = True
PERSISTENT = True

MAX_LEN = 384
STRIDE = 192
DROPOUT_RATE = 0.3 # 모델 구조 정의 시 필요
MODEL_NAME = "beomi/KcELECTRA-base-v2022"

print(f"\n📋 Config:")
print(f"  AMP dtype: {amp_dtype}")
print(f"  MAX_LEN/STRIDE: {MAX_LEN}/{STRIDE}")
print(f"  Model Path: {SAVE_DIR}")
print(f"  Data Path: {DATA_DIR}")

PyTorch: 2.8.0+cu126 | CUDA: 12.6
GPU: NVIDIA A100-SXM4-80GB
Python: 3.12.12
Device: cuda

📋 Config:
  AMP dtype: torch.bfloat16
  MAX_LEN/STRIDE: 384/192
  Model Path: /content/drive/MyDrive/project1/kcelectra_mtl_ckpt_v3
  Data Path: /content/drive/MyDrive/project1/Processed_Data/MTL_ready_v3


## 3. 모델 클래스 정의

학습 스크립트와 **정확히 동일한** `ElectraMTL` 클래스를 정의해야 `state_dict`를 로드할 수 있습니다.

In [3]:
class ElectraMTL(nn.Module):
    def __init__(self, base_name: str, n_u: int = 3, n_s: int = 4, dropout=0.3):
        super().__init__()
        self.base = AutoModel.from_pretrained(base_name)
        hdim = self.base.config.hidden_size
        self.dropout = nn.Dropout(dropout)
        self.head_u = nn.Linear(hdim, n_u)
        self.head_s = nn.Linear(hdim, n_s)

    def forward(self, input_ids, attention_mask):
        out = self.base(input_ids=input_ids, attention_mask=attention_mask)
        cls = out.last_hidden_state[:, 0]
        z = self.dropout(cls)
        return self.head_u(z), self.head_s(z)

## 4. 토크나이저 및 모델 로드

In [4]:
print(f"\n🔤 Loading tokenizer: {MODEL_NAME}")
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME, use_fast=True)

print("🏗️ Building model structure...")
model = ElectraMTL(MODEL_NAME, n_u=3, n_s=4, dropout=DROPOUT_RATE).to(device)
model.eval() # 추론 모드로 설정

ckpt_path = SAVE_DIR / "best_mtl.pt"
if ckpt_path.exists():
    try:
        print(f"📥 Loading weights from {ckpt_path}")
        state = torch.load(ckpt_path, map_location=device, weights_only=False)
        model.load_state_dict(state["state_dict"])
        print(f"✅ Model loaded successfully from epoch {state.get('epoch', 'N/A')}")
        print(f"   Best combo score: {state.get('best_combo', 0.0):.4f}")

        # 저장된 config 로드 (참고용)
        loaded_config = state.get("config", {})
        print(f"   Loaded Config (ML/ST): {loaded_config.get('max_len', 'N/A')}/{loaded_config.get('stride', 'N/A')}")
        # 현재 설정과 저장된 설정이 다를 경우 경고
        if loaded_config.get('max_len', MAX_LEN) != MAX_LEN or loaded_config.get('stride', STRIDE) != STRIDE:
            print("⚠️ WARNING: Current MAX_LEN/STRIDE differs from saved model config!")

    except Exception as e:
        print(f"❌ Error loading model: {e}")
else:
    print(f"❌ ERROR: Model checkpoint not found at {ckpt_path}")
    print("Please check the SAVE_DIR path or train the model first.")


🔤 Loading tokenizer: beomi/KcELECTRA-base-v2022


tokenizer_config.json:   0%|          | 0.00/288 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/504 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

🏗️ Building model structure...


pytorch_model.bin:   0%|          | 0.00/511M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/511M [00:00<?, ?B/s]

❌ ERROR: Model checkpoint not found at /content/drive/MyDrive/project1/kcelectra_mtl_ckpt_v3/best_mtl.pt
Please check the SAVE_DIR path or train the model first.


## 5. 헬퍼 함수 정의 (데이터 로드 및 처리)

성능 검증을 위해 테스트 데이터를 로드하고 학습 시와 동일하게 처리합니다.

In [5]:
# 1. 데이터 로드 헬퍼
def read_any(base: Path, stem: str):
    """Parquet → CSV → Feather 순으로 시도"""
    for ext, fn in [(".parquet", pd.read_parquet),
                    (".csv", pd.read_csv),
                    (".feather", pd.read_feather)]:
        fp = base / f"{stem}{ext}"
        if fp.exists():
            print(f"  → {fp.name}")
            try:
                return fn(fp)
            except Exception as e:
                print(f"⚠️ Failed to read {fp.name}: {e}. Try installing engines or checking versions.")
    raise FileNotFoundError(f"{stem} not found in {base}")

def squeeze_label(obj, key=None):
    if isinstance(obj, pd.DataFrame):
        if key and key in obj.columns:
            return obj[key].astype(int)
        if obj.shape[1] == 1:
            return obj.iloc[:, 0].astype(int)
    return pd.Series(obj).astype(int)

# 2. 청크 캐시 헬퍼
def build_chunk_cache_MTL(texts, y_u, y_s, tokenizer, split_name: str,
                          max_len=512, stride=256, batch_size=1024):
    """청크 단위로 토크나이징 + 캐싱"""
    cache_path = CACHE_DIR / f"{split_name}_ml{max_len}_st{stride}.pt"

    if cache_path.exists():
        try:
            print(f"⚡ Loading cached chunks: {cache_path.name}")
            cached = torch.load(cache_path, weights_only=False)
            expected_keys = {"input_ids", "attention_mask", "y_u", "y_s", "sample_idx"}
            if expected_keys.issubset(cached.keys()):
                if cached["input_ids"].dtype == torch.int32 and cached["attention_mask"].dtype == torch.int8:
                    print(f"  Chunks: {cached['y_u'].size(0):,}")
                    return cached
                else:
                    print("  ⚠️ Cache dtype/shape invalid, regenerating...")
            else:
                print("  ⚠️ Invalid cache, regenerating...")
        except Exception as e:
            print(f"  ⚠️ Cache load failed: {e}, regenerating...")

    print(f"🔨 Building cache: {split_name} (MAX_LEN={max_len}, STRIDE={stride})")
    all_ids, all_masks, all_u, all_s, all_map = [], [], [], [], []
    N = len(texts)

    for i in tqdm(range(0, N, batch_size), desc=f"Tokenize [{split_name}]"):
        batch_texts = [str(t) for t in texts[i:i+batch_size]]

        enc = tokenizer(
            batch_texts,
            max_length=max_len,
            truncation=True,
            padding="max_length",
            return_attention_mask=True,
            return_overflowing_tokens=True,
            stride=stride,
            return_tensors="pt"
        )

        mapping = enc.pop("overflow_to_sample_mapping")
        global_map = (mapping + i).to(torch.int32)

        ids = enc["input_ids"].to(torch.int32)
        mask = enc["attention_mask"].to(torch.int8)

        labs_u = torch.tensor([int(y_u.iloc[int(j)]) for j in global_map],
                              dtype=torch.int64)
        labs_s = torch.tensor([int(y_s.iloc[int(j)]) for j in global_map],
                              dtype=torch.int64)

        all_ids.append(ids)
        all_masks.append(mask)
        all_u.append(labs_u)
        all_s.append(labs_s)
        all_map.append(global_map)

    tensors = {
        "input_ids": torch.cat(all_ids, 0),
        "attention_mask": torch.cat(all_masks, 0),
        "y_u": torch.cat(all_u, 0),
        "y_s": torch.cat(all_s, 0),
        "sample_idx": torch.cat(all_map, 0),
    }

    torch.save(tensors, cache_path)
    print(f"💾 Saved cache → {cache_path.name} | Chunks: {tensors['y_u'].size(0):,}")
    return tensors

# 3. Dataset 클래스
class MTLChunkDataset(torch.utils.data.Dataset):
    def __init__(self, tensors):
        self.t = tensors

    def __len__(self):
        return self.t["y_u"].size(0)

    def __getitem__(self, i):
        return {
            "input_ids": self.t["input_ids"][i].long(),
            "attention_mask": self.t["attention_mask"][i].bool(),
            "y_u": self.t["y_u"][i],
            "y_s": self.t["y_s"][i],
        }

# 4. DataLoader 생성 헬퍼
def make_loader(ds, batch_size, shuffle):
    try:
        return DataLoader(
            ds, batch_size=batch_size, shuffle=shuffle,
            num_workers=NUM_WORKERS, pin_memory=PIN_MEMORY,
            prefetch_factor=PREFETCH, persistent_workers=PERSISTENT
        )
    except Exception as e:
        print(f"⚠️ DataLoader fallback (workers=0): {e}")
        return DataLoader(ds, batch_size=batch_size, shuffle=shuffle,
                         num_workers=0, pin_memory=False)

## 6. 테스트 데이터 로드 및 처리

성능 검증을 위해 `DATA_DIR`에서 테스트셋(`X_test`, `y_test_...`)을 로드합니다.

In [6]:
print("\n📂 Loading TEST data...")
try:
    X_test  = read_any(DATA_DIR, "X_test")
    y_u_te  = read_any(DATA_DIR, "y_test_urgency")
    y_s_te  = read_any(DATA_DIR, "y_test_sentiment")

    y_u_te = squeeze_label(y_u_te, "y_urgency")
    y_s_te = squeeze_label(y_s_te, "y_sentiment")
    text_te = X_test["full_text"].astype(str)

    print(f"✅ Test data loaded: {len(text_te):,}")

    print("\n🔄 Preparing test chunks...")
    va_cache = build_chunk_cache_MTL(text_te, y_u_te, y_s_te, tokenizer, "val", MAX_LEN, STRIDE)

    print("\n🚚 Creating Test DataLoader...")
    val_loader = make_loader(MTLChunkDataset(va_cache), BATCH_SIZE * 2, False)
    print(f"  Test batches: {len(val_loader):,}")

except FileNotFoundError as e:
    print(f"❌ ERROR: Test data not found in {DATA_DIR}. {e}")
    print("Performance check cannot proceed. Skipping to inference demo.")
    val_loader = None
except Exception as e:
    print(f"❌ ERROR loading data: {e}")
    val_loader = None


📂 Loading TEST data...
  → X_test.parquet
  → y_test_urgency.parquet
  → y_test_sentiment.parquet
✅ Test data loaded: 25,436

🔄 Preparing test chunks...
🔨 Building cache: val (MAX_LEN=384, STRIDE=192)


Tokenize [val]:   0%|          | 0/25 [00:00<?, ?it/s]

💾 Saved cache → val_ml384_st192.pt | Chunks: 39,466

🚚 Creating Test DataLoader...
  Test batches: 124


## 7. 평가 함수 정의

학습 스크립트의 `evaluate_call_level` 함수를 그대로 가져와 성능을 측정합니다.

In [7]:
@torch.no_grad()
def evaluate_call_level(model, loader, cache_tensors):
    """Call-level 평가 (청크 로짓 평균)"""
    if loader is None:
        print("Loader is not available. Skipping evaluation.")
        return None

    model.eval()
    logits_u_list, logits_s_list = [], []
    labels_u_list, labels_s_list = [], []

    for batch in tqdm(loader, desc="Eval", leave=False):
        ids = batch["input_ids"].to(device, non_blocking=True)
        mask = batch["attention_mask"].to(device, non_blocking=True)
        yu = batch["y_u"].to(device, non_blocking=True)
        ys = batch["y_s"].to(device, non_blocking=True)

        with autocast('cuda', dtype=amp_dtype, enabled=torch.cuda.is_available()):
            lu, ls = model(ids, mask)

        # FP32 캐스팅 (넘파이 변환 전)
        logits_u_list.append(lu.detach().float().cpu().numpy())
        logits_s_list.append(ls.detach().float().cpu().numpy())
        labels_u_list.append(yu.detach().cpu().numpy())
        labels_s_list.append(ys.detach().cpu().numpy())

    # 연결
    all_logits_u = np.concatenate(logits_u_list, 0)
    all_logits_s = np.concatenate(logits_s_list, 0)
    all_labels_u = np.concatenate(labels_u_list, 0)
    all_labels_s = np.concatenate(labels_s_list, 0)

    idx = cache_tensors["sample_idx"].cpu().numpy()

    # Call-level 집계
    results = {}
    for task_name, logits, ytrue in [
        ("urgency", all_logits_u, all_labels_u),
        ("sentiment", all_logits_s, all_labels_s)
    ]:
        preds, gts = [], []
        for sid in np.unique(idx):
            mask = (idx == sid)
            pooled_logits = logits[mask].mean(0)
            preds.append(pooled_logits.argmax())
            gts.append(ytrue[mask][0])

        preds = np.array(preds)
        gts = np.array(gts)

        results[task_name] = {
            "micro": f1_score(gts, preds, average="micro"),
            "macro": f1_score(gts, preds, average="macro"),
            "weighted": f1_score(gts, preds, average="weighted"),
            "per_class": f1_score(gts, preds, average=None).tolist(),
            "predictions": preds,
            "ground_truth": gts,
            "report": classification_report(gts, preds, output_dict=True)
        }

    return results

## 8. 📊 테스트셋 성능 검증 실행

로드된 `best_mtl.pt` 모델을 사용하여 테스트셋에 대한 성능을 확인합니다.

In [8]:
print("\n📊 Final Evaluation on Test Set (using loaded model):")
final_results = evaluate_call_level(model, val_loader, va_cache)

if final_results:
    print(f"\n{'='*70}")
    print("         TEST SET PERFORMANCE REPORT")
    print(f"{'='*70}")

    print("\nUrgency:")
    print(f"  Weighted F1: {final_results['urgency']['weighted']:.4f}")
    print(f"  Per-class F1: {final_results['urgency']['per_class']}")
    print("  Classification Report (Dict):")
    print(final_results['urgency']['report'])

    print("\nSentiment:")
    print(f"  Weighted F1: {final_results['sentiment']['weighted']:.4f}")
    print(f"  Per-class F1: {final_results['sentiment']['per_class']}")
    print("  Classification Report (Dict):")
    print(final_results['sentiment']['report'])

    print("\n📈 Confusion Matrix (Urgency):")
    cm_u = confusion_matrix(final_results['urgency']['ground_truth'],
                            final_results['urgency']['predictions'])
    print(cm_u)

    print("\n📈 Confusion Matrix (Sentiment):")
    cm_s = confusion_matrix(final_results['sentiment']['ground_truth'],
                            final_results['sentiment']['predictions'])
    print(cm_s)


📊 Final Evaluation on Test Set (using loaded model):


Eval:   0%|          | 0/124 [00:00<?, ?it/s]


         TEST SET PERFORMANCE REPORT

Urgency:
  Weighted F1: 0.3155
  Per-class F1: [0.3571111604171289, 0.21959078459803447, 0.372491434165443]
  Classification Report (Dict):
{'0': {'precision': 0.35691318327974275, 'recall': 0.3573093573093573, 'f1-score': 0.3571111604171289, 'support': 9009.0}, '1': {'precision': 0.35430205354821936, 'recall': 0.15909886774833665, 'f1-score': 0.21959078459803447, 'support': 8567.0}, '2': {'precision': 0.3027048528241846, 'recall': 0.48409669211195927, 'f1-score': 0.372491434165443, 'support': 7860.0}, 'accuracy': 0.32972951721968863, 'macro avg': {'precision': 0.33797336321738225, 'recall': 0.3335016390565511, 'f1-score': 0.31639779306020216, 'support': 25436.0}, 'weighted avg': {'precision': 0.3392827765416295, 'recall': 0.32972951721968863, 'f1-score': 0.31554613022447153, 'support': 25436.0}}

Sentiment:
  Weighted F1: 0.0082
  Per-class F1: [0.0, 0.0, 0.0, 0.12370449599822964]
  Classification Report (Dict):
{'0': {'precision': 0.0, 'recall':

## 9. 추론 함수 정의 (새 텍스트용)

학습 스크립트의 `predict_text` 함수를 그대로 가져와 새로운 텍스트에 대한 추론을 수행합니다.

In [9]:
@torch.no_grad()
def predict_text(texts, aggregate="mean"):
    """긴 텍스트 예측 (청크 평균)"""
    if isinstance(texts, str):
        texts = [texts]

    model.eval()
    results = []

    for text in texts:
        enc = tokenizer(
            text,
            max_length=MAX_LEN,
            padding="max_length",
            truncation=True,
            return_tensors="pt",
            return_overflowing_tokens=True,
            stride=STRIDE
        ).to(device)

        with autocast('cuda', dtype=amp_dtype, enabled=torch.cuda.is_available()):
            lu, ls = model(
                enc["input_ids"],
                enc["attention_mask"]
            )
            prob_u = lu.softmax(-1).float()  # FP32 캐스팅
            prob_s = ls.softmax(-1).float()  # FP32 캐스팅

        # 청크 평균
        if aggregate == "mean":
            avg_u = prob_u.mean(0)
            avg_s = prob_s.mean(0)
        else:  # max (혹은 다른 집계 방식)
            avg_u = prob_u.max(0).values
            avg_s = prob_s.max(0).values

        pred_u = int(avg_u.argmax().item())
        pred_s = int(avg_s.argmax().item())
        conf_u = float(avg_u.max().item())
        conf_s = float(avg_s.max().item())

        results.append({
            "urgency": pred_u,
            "sentiment": pred_s,
            "urgency_conf": conf_u,
            "sentiment_conf": conf_s
        })

    return results[0] if len(results) == 1 else results

## 10. 🧪 추론 데모

임의의 텍스트를 입력하여 모델의 예측을 확인합니다.

In [10]:
print("\n🧪 Test Inference:")

# (라벨 맵 참고: Urgency={0:'하', 1:'중', 2:'상'} / Sentiment={0:'anxious', 1:'angry', 2:'neutral', 3:'sad'})

text_1 = "[TURN=1] [CALLER] 아 네 안녕하세요 [TURN=2] [AGENT] 네 안녕하세요 [TURN=3] [CALLER] 저기요 제가 지금 손가락을 좀 다쳤는데 피가 너무 많이 나가지고요 [TURN=4] [AGENT] 네 손가락 어디 다치셨어요 [TURN=5] [CALLER] 지금 그 베여가지고 피가 멈추질 않아요 너무 아파요"
text_2 = "[TURN=1] [CALLER] 저기 차가 방전이 돼서 그러는데요 [TURN=2] [AGENT] 아 네 차량 방전이요 위치가 어디시죠 [TURN=3] [CALLER] 여기가 그 강남역 3번 출구 앞이거든요 [TURN=4] [AGENT] 네 알겠습니다 금방 기사님 보내드릴게요 [TURN=5] [CALLER] 네 감사합니다"

print("--- Sample 1 (긴급/불안) ---")
result_1 = predict_text(text_1)
print(f"Input: {text_1[:100]}...")
print(f"Prediction: {result_1}")

print("--- Sample 2 (비긴급/평정) ---")
result_2 = predict_text(text_2)
print(f"Input: {text_2[:100]}...")
print(f"Prediction: {result_2}")

print("\n✅ Inference Demo Done!")


🧪 Test Inference:
--- Sample 1 (긴급/불안) ---
Input: [TURN=1] [CALLER] 아 네 안녕하세요 [TURN=2] [AGENT] 네 안녕하세요 [TURN=3] [CALLER] 저기요 제가 지금 손가락을 좀 다쳤는데 피가 너무 많...
Prediction: {'urgency': 2, 'sentiment': 3, 'urgency_conf': 0.36568742990493774, 'sentiment_conf': 0.34772613644599915}
--- Sample 2 (비긴급/평정) ---
Input: [TURN=1] [CALLER] 저기 차가 방전이 돼서 그러는데요 [TURN=2] [AGENT] 아 네 차량 방전이요 위치가 어디시죠 [TURN=3] [CALLER] 여기가 그 강...
Prediction: {'urgency': 1, 'sentiment': 3, 'urgency_conf': 0.34620898962020874, 'sentiment_conf': 0.3360721468925476}

✅ Inference Demo Done!
